
# The `require_recipient` function

The function is declared in `action.codon` as follows:

```python
def require_recipient(account: Name)
```

The `require_recipient` function is utilized to notify other contracts. If the `account` contract has the same action, this action will be called.

The following code in `sender.codon` and `receiver.codon` demonstrates how to send a notification from one contract to another.

In [6]:
!mkdir -p tests/notify

In [7]:
%%writefile tests/notify/sender.codon

from chain.name import Name
from chain.contract import Contract
from chain.action import require_recipient

@contract(main=True)
class MyContract(Contract):

    @action('sayhello')
    def sayhello(self, receiver: Name):
        print('hello, world')
        require_recipient(receiver)

Writing notify/sender.codon


In [8]:
%%writefile tests/notify/receiver.codon

from chain.name import Name
from chain.contract import Contract

@contract(main=True)
class MyContract(Contract):

    @action('sayhello', notify=True)
    def sayhello(self, receiver: Name):
        assert not self.receiver == self.first_receiver
        assert receiver == self.receiver
        print('hello, world from notify')

Writing notify/receiver.codon


Note that the definition of the `sayhello` function in `receiver.codon` is slightly different from that in `sender.codon`. The `notify=True` in the `action` decorator of the `sayhello` function in `receiver.codon` is used to specify that this action is used to receive notifications and can only be triggered by calling `require_recipient`.

The following is the test code:

In [18]:
%%writefile tests/test.py

import os
from ipyeos.chaintester import ChainTester

from pscdk.tester import update_auth, logger

def init_notify(dir_name=''):
    t = ChainTester(True)
    update_auth(t, 'hello')

    wasm_file = os.path.join(dir_name, 'notify/sender.wasm')
    with open(wasm_file, 'rb') as f:
        code = f.read()
    abi_file = os.path.join(dir_name, 'notify/sender.abi')
    with open(abi_file, 'r') as f:
        abi = f.read()
    t.deploy_contract('hello', code, abi)
    t.produce_block()

    wasm_file = os.path.join(dir_name, 'notify/receiver.wasm')
    with open(wasm_file, 'rb') as f:
        code = f.read()
    abi_file = os.path.join(dir_name, 'notify/receiver.abi')
    with open(abi_file, 'r') as f:
        abi = f.read()
    t.deploy_contract('alice', code, abi)
    t.produce_block()
    return t

def test_notify():
    t = init_notify()
    args = {'receiver': 'alice'}
    ret = t.push_action('hello', 'sayhello', args, {'hello': 'active'})
    t.produce_block()
    logger.info("++++++++++%s\n", ret['elapsed'])

Overwriting test.py


Compile:

In [10]:
%%bash
python-contract build notify/receiver.codon
python-contract build notify/sender.codon

 class.cpp 249 +++++++class name: MyContract
 class.cpp 26 generating apply method for class MyContract
 class.cpp 53 +++++++fn.name: sayhello
 typecheck.cpp 76 +++++++ctx->cache->classRealizationCnt: 527
 compiler.cpp 149 Generating abi file: notify/receiver.abi
 llvisitor.cpp 495 ++++++++++executable dir:/Users/newworld/dev/github/codon/build


/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/pscdk/codon/bin/codon build --release  --march=wasm32 --linker-flags=" -L/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/pscdk/codon/lib/codon -lcodonrt-wasm32 -L/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/pscdk/codon/lib -lc++ -lc" notify/receiver.codon


 class.cpp 249 +++++++class name: MyContract
 class.cpp 26 generating apply method for class MyContract
 class.cpp 53 +++++++fn.name: sayhello
 typecheck.cpp 76 +++++++ctx->cache->classRealizationCnt: 511
 compiler.cpp 149 Generating abi file: notify/sender.abi
 llvisitor.cpp 495 ++++++++++executable dir:/Users/newworld/dev/github/codon/build


/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/pscdk/codon/bin/codon build --release  --march=wasm32 --linker-flags=" -L/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/pscdk/codon/lib/codon -lcodonrt-wasm32 -L/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/pscdk/codon/lib -lc++ -lc" notify/sender.codon


Test:

In [19]:
!ipyeos -m pytest -s -x tests/test.py -k test_notify

export CHAIN_API_LIB=/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/ipyeos/release/lib/libchain_api.dylib
export VM_API_LIB=/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/ipyeos/release/lib/libvm_api.dylib
export PYTHON_SHARED_LIB_PATH=/Users/newworld/miniconda3/envs/chat/lib/libpython3.10.dylib
/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/ipyeos/release/bin/ipyeos -m pytest -s -x test.py -k test_notify
============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-7.2.0, pluggy-1.0.0
benchmark: 4.0.0 (defaults: timer=time.perf_counter disable_gc=False min_rounds=5 min_time=0.000005 max_time=1.0 calibration_precision=10 warmup=False warmup_iterations=100000)
rootdir: /Users/newworld/dev/github/pscdk-book/notebook
plugins: asyncio-0.20.3, integration-0.2.3, dash-2.9.3, mock-3.10.0, cov-4.0.0, anyio-3.6.2, benchmark-4.0.0
asyncio: mode=strict
collected 1 item             

info  2023-05-12T08:17:18.323 thread-0  platform_timer_accurac:62     compute_and_print_ti ] Checktime timer accuracy: min:4us max:366us mean:22us stddev:13us
warn  2023-05-12T08:17:18.329 thread-0  controller.cpp:605            startup              ] No existing chain state or fork database. Initializing fresh blockchain state and resetting fork database.
warn  2023-05-12T08:17:18.329 thread-0  controller.cpp:457            initialize_blockchai ] Initializing new blockchain with genesis state
info  2023-05-12T08:17:18.338 thread-0  controller.cpp:530            replay               ] no irreversible blocks need to be replayed
info  2023-05-12T08:17:18.338 thread-0  controller.cpp:553            replay               ] replayed 0 blocks in 0 seconds, 0.00000000000954606 ms/block
debug 2023-05-12T08:17:18.339 thread-0  controller.cpp:2444           clear_expired_input_ ] removed 0 expired transactions of the 0 input dedup list, pending block time 2018-06-01T12:00:00.500
debug 2023-05-12T

Output:

```
[(hello,sayhello)->hello]: CONSOLE OUTPUT BEGIN =====================
hello, world

[(hello,sayhello)->hello]: CONSOLE OUTPUT END   =====================
debug 2023-03-28T13:08:01.110 thread-0  apply_context.cpp:30          print_debug          ] 
[(hello,sayhello)->alice]: CONSOLE OUTPUT BEGIN =====================
hello, world from notify

[(hello,sayhello)->alice]: CONSOLE OUTPUT END   =====================
```